# 세종시 주택시장 분석
## 팀명: 세종대왕

- - -

2012년 새롭게 출범한 세종특별자치시는, **'행정중심복합도시'** 라는 우리나라에서 처음으로 시도하는 특별한 형태의 도시입니다.   
또한 작년, '세종천도설' 등 여러 이슈에 휩싸이며 주택의 거래량이 가장 빠르게 증가하던 곳이기도 합니다.   
그래서 저희는 주어진 데이터를 통해 **세종시의 주택시장이 어떤 특징을 가지고 있는지**를 중심으로 명료한 시각화 모델을 통해 제시하고자 합니다. 

- - -

## 코드 목차

### 0. Import
  
### 1. Data Preprocessing  
  
- 내부 데이터 전처리    
- 외부 데이터 전처리    
  
### 2. Visualization  
  
- 시기 별 이슈 확인 (WordCloud)  
- 주택 및 거래 관련  
- 인구 관련
  
### 3. Modeling   

- 지수 생성  
- LDA  
- Multiple Regression Analysis  
- Oaxaca Decomposition  
- - -

# 0. Import

In [ ]:
from geoband.API import *
import folium
import json
import geopandas as gpd
import pandas as pd
pd.options.display.float_format = '{:.5f}'.format
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import pydeck as pdk
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.io as pio
pio.renderers.default = "notebook_connected"
import warnings
warnings.filterwarnings("ignore")
col = px.colors.qualitative.Pastel

from geopandas import GeoDataFrame
from pandas import DataFrame
import deckgljupyter.Layer as deckgl

from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.cluster import AgglomerativeClustering
import random
from sklearn.preprocessing import MinMaxScaler
np.random.seed(seed=42)
from sklearn.decomposition import PCA

random.seed(42)
import hdbscan

In [ ]:
GetCompasData('SBJ_2102_001', '2', '2.세종시_표제부.csv')
GetCompasData('SBJ_2102_001', '3', '3.세종시_아파트(매매)_실거래가.csv')
GetCompasData('SBJ_2102_001', '4', '4.세종시_연립다세대(매매)_실거래가.csv')
GetCompasData('SBJ_2102_001', '5', '5.세종시_단독다가구(매매)_실거래가.csv')
GetCompasData('SBJ_2102_001', '6', '6.세종시_오피스텔(매매)_실거래가.csv')
GetCompasData('SBJ_2102_001', '7', '7.세종시_아파트(전월세)_실거래가.csv')
GetCompasData('SBJ_2102_001', '8', '8.세종시_연립다세대(전월세)_실거래가.csv')
GetCompasData('SBJ_2102_001', '9', '9.세종시_단독다가구(전월세)_실거래가.csv')
GetCompasData('SBJ_2102_001', '10', '10.세종시_오피스텔(전월세)_실거래가.csv')
GetCompasData('SBJ_2102_001', '14', '14.세종시_상권정보.csv')
GetCompasData('SBJ_2102_001', '18', '18.세종시_개별공시지가(2017~2020).csv')
GetCompasData('SBJ_2102_001', '19', '19.세종시_연령별_거주인구정보_격자.geojson')
GetCompasData('SBJ_2102_001', '20', '20.세종시_전입자수.csv')
GetCompasData('SBJ_2102_001', '21', '21.세종시_전출자수.csv')
GetCompasData('SBJ_2102_001', '22', '22.세종시_연령별_인구현황.csv')
GetCompasData('SBJ_2102_001', '28', '28.세종시_지역별_세대원수별_세대수.csv')
GetCompasData('SBJ_2102_001', '29', '29.세종시_거주의사(향후).csv')
GetCompasData('SBJ_2102_001', '31', '31.세종시_법정경계(읍면동).geojson')
GetCompasData('SBJ_2102_001', '32', '32.세종시_행정경계(읍면동).geojson')

In [ ]:
data2 = pd.read_csv('2.세종시_표제부.csv',encoding='utf-8')
data3 = pd.read_csv('3.세종시_아파트(매매)_실거래가.csv', encoding='utf-8')
data4 = pd.read_csv('4.세종시_연립다세대(매매)_실거래가.csv',  encoding='utf-8')
data5 = pd.read_csv('5.세종시_단독다가구(매매)_실거래가.csv', encoding='utf-8')
data6 = pd.read_csv('6.세종시_오피스텔(매매)_실거래가.csv',  encoding='utf-8')
data7 = pd.read_csv('7.세종시_아파트(전월세)_실거래가.csv',  encoding='utf-8')
data8 = pd.read_csv('8.세종시_연립다세대(전월세)_실거래가.csv', encoding='utf-8')
data9 = pd.read_csv('9.세종시_단독다가구(전월세)_실거래가.csv', encoding='utf-8')
data10 = pd.read_csv('10.세종시_오피스텔(전월세)_실거래가.csv', encoding='utf-8')
data14 = pd.read_csv('14.세종시_상권정보.csv', encoding= 'utf-8')
data18 = pd.read_csv('18.세종시_개별공시지가(2017~2020).csv', encoding='utf-8')
data19 = gpd.read_file('19.세종시_연령별_거주인구정보_격자.geojson')
data20 = pd.read_csv('20.세종시_전입자수.csv', encoding='utf-8')
data21 = pd.read_csv('21.세종시_전출자수.csv', encoding='utf-8')
data22 = pd.read_csv('22.세종시_연령별_인구현황.csv', encoding='utf-8')
data28 = pd.read_csv('28.세종시_지역별_세대원수별_세대수.csv', encoding='utf-8')
data29 = pd.read_csv('29.세종시_거주의사(향후).csv', encoding='utf-8')
data31 = gpd.read_file('31.세종시_법정경계(읍면동).geojson')
data32 = gpd.read_file('32.세종시_행정경계(읍면동).geojson')
geo = gpd.read_file('final_geo.geojson')

- - -
# 1. Data Preprocessing

## 1) 매매, 전월세, 공시지가 데이터 전처리

- 지번주소가 다른 방식(ex. 본번, 부번)으로 제공되어있는 

  매매(3,4,5,6), 전월세(7,8,9,10), 공시지가(18) raw 데이터를 동일한 지번주소로 통합.

In [ ]:
# 시군구, 본번, 부번 합쳐서 지번주소 만드는 함수 정의

def get_address(data):
    data = data.reset_index()
    df = data
    df['부번']=df['부번'].astype(str)
    df['본번']=df['본번'].astype(str)
    df['시군구']=df['시군구'].astype(str)
    
    drop = df.loc[(df['본번'] == '0') & (df['부번'] == '0')].index
    df = df.drop(drop)
    
    # 유형 별로 주소 데이터 나누기
    df1 = df[df['본번'] == '0'][['index','시군구','본번','부번']]
    df2 = df[df['부번'] == '0'][['index','시군구','본번','부번']]
    df3 = df.loc[(df['본번'] != '0') & (df['부번'] != '0')][['index','시군구','본번','부번']]
    
    # 각 유형에 따라 번지 부여
    df1['입력주소'] = " " + df1['시군구'] + " " + df1['부번'] + "번지"
    df2['입력주소'] = " " + df2['시군구'] + " " + df2['본번'] + "번지"
    df3['입력주소'] = " " + df3['시군구']+ " " + df3['본번'] + "-" + df3['부번'] + "번지"
    
    # 3개의 데이터 Merging
    from functools import reduce
    dfs = [df1,df2,df3]
    df_merge = reduce(lambda left, right: 
                      pd.merge(left, right, on=['index','입력주소'], how='outer'), dfs)
    df_merge = df_merge[['index','입력주소']]
    df_final = pd.merge(data,df_merge,on = 'index',how='outer')
    return df_final

### 매매, 전월세 데이터 지번주소 뽑아내기

In [ ]:
# 각 데이터에서 지번 주소들 뽑아내기

df1 = get_address(data3)[['index','입력주소']]
df2 = get_address(data4)[['index','입력주소']]
df3 = get_address(data6)[['index','입력주소']]
df4 = get_address(data7)[['index','입력주소']]
df5 = get_address(data8)[['index','입력주소']]
df6 = get_address(data10)[['index','입력주소']]

# Data Merging
from functools import reduce
dfs = [df1,df2,df3,df4,df5,df6]
df_merge = reduce(lambda left, right: pd.merge(left, right, on=['index','입력주소'], how='outer'), dfs)

df_merge['happy'] = 1
df_merge = df_merge[['입력주소','happy']]
address = df_merge.drop_duplicates().reset_index()

address.to_csv('address.csv', encoding='utf-8') # Geocoding 할 수 있도록 데이터 저장

### 공시지가 데이터에서 주택만 추출

- 2번 표제부 데이터에서 아파트, 단독주택 데이터만 추출하여 새로운 변수로 저장

In [ ]:
# 공시지가 데이터에서 입력주소, 주소 코드 만들어주기

data18['지번'] = data18['지번'].str.replace("_","-")  # 지번 바꾸기
data18['입력주소'] = data18['법정동명'] + " " + data18['지번'] + "번지"
data18['법정동코드'] = data18['법정동코드'].astype(str)
data18['code'] = data18['법정동코드'] + '/' + data18['지번']

In [ ]:
#표제부에서 주택(아파트, 단독) 에 해당하는 지번만 추출하기

data2['주용도코드명'] = data2['주용도코드명'].fillna('결측치')
house = data2[data2['주용도코드명'].str.contains('주택')] #표제부에서 주택만 추출
house['지'] = house['지'].astype(str)
house['번'] = house['번'].astype(str)
house['법정동코드'] = house['법정동코드'].astype(str)
house['시군구코드'] = house['시군구코드'].astype(str)

# 지번 변수 만들기
drop = house.loc[(house['지'] == '0') & (house['번'] == '0')].index
house = house.drop(drop)
house1 = house[house['지'] == '0']
house2 = house[house['번'] == '0']
house3 = house.loc[(house['지'] != '0') & (house['번'] != '0')]
house1['지번'] = house1['번']
house3['지번'] = house3['번'] + '-' + house3['지']

# Data Merging
house = pd.concat([house1,house3])
house = house[['대지위치','건물명','주용도코드명','시군구코드','법정동코드','지번','사용승인일']]
house['code'] = house['시군구코드']+ house['법정동코드'] + '/' + house['지번'] #코드 만들기
house = house[['대지위치','주용도코드명','건물명','code','사용승인일']]

In [ ]:
# 표제부 데이터와 공시지가 데이터 합쳐주기 

houseonly = pd.merge(data18, house, on = 'code') #주택만 추출된 데이터!
houseonly = houseonly[['건물명','법정동명','기준년도','공시지가','대지위치','주용도코드명','사용승인일']]
houseonly.to_csv('houseonly.csv', encoding='utf-8') #Geocoding 위해서 저장

## 2) Geocoding 된 데이터들 격자에 부여

- Geocoding Tool을 활용하여, 지번주소를 위경도 좌표 데이터로 변환하는 plus_grid 함수 정의

In [ ]:
# 위경도 데이터를 격자로 부여하는 함수 정의

def plus_grid(df, geo, X, Y, drop=True):
    data1= gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df[X], df[Y])) #X는 경도, Y는 위도
    merge_data = gpd.sjoin(data1, geo, op = "within", how = "right")
    merge_data.crs = geo.crs
    if drop:
        merge_data = merge_data.dropna()
    return merge_data

###  버스정류장 데이터

- 세종시 버스정류장 관련 외부 데이터를 활용하여, 하루 배차 회수 1 이하, 정보 누락된 버스정류장 NODE 제외.

- plus_grid 함수 사용하여, 그리드 별 개수 COUNT 후 세종시 버스 종류인 BRT, 광역, 지선, 간선, 마을 버스를 column으로 지정



In [ ]:
BUS = pd.read_csv("sejong_busnode.csv", encoding='utf-8') # 버스정류장 데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

TIME=pd.read_csv("time.csv", encoding='utf-8') # 버스 종류별 배차 횟수 데이터버스 종류별 배차 횟수 데이터

BUS=BUS.dropna(how="all")

#'정류장번호 없음'은 삭제함.
index1=BUS[BUS['NODENO']=="정류장번호 없음"].index
BUS=BUS.drop(index1)
# 정류장 번호 '0'은 삭제함
index2=BUS[BUS['NODENO']=="0"].index
BUS=BUS.drop(index2)
# index 새롭게 지정
BUS=BUS.reset_index()

for i in range(0,len(TIME)): # 버스에 따라, 배차 횟수를 hap이라는 열에 지정.
    for j in range(0,len(BUS)) :
        if BUS['BUS_NUM'][j] == TIME['버스이름'][i]:
            BUS['hap'][j] = TIME['개수'][i]

# PCA를 통해, 지수를 생성할 것인데 이때의 변수를 버스 종류로 할 것이기에
# BUS 데이터에 빈 열을 생성해주었다.
BUS['BRT']=""
BUS['광역버스']=""
BUS['지선버스']=""
BUS['간선버스']=""
BUS['마을버스']=""

BRT=np.array(['B2','900','B5'],dtype=object)
광역버스=np.array(['1000','1001','1004','1005'],dtype=object)
지선버스=np.array(['11','12','13','52','53','61','201','203','221','300','910'],dtype=object)
간선버스=np.array(['430','991'],dtype=object)
마을버스=np.array(['16','17','31','32','33','35','62','63','64','65','66','661','67','69','691','71','72','73','74','75','76','81','82','83','84','85','86','91','92','93','94','95'],dtype=object)

for i in range(0,len(BUS)):
    if BUS["BUS_NUM"][i] in BRT:
        BUS['BRT'][i]=BUS['hap'][i]
    elif BUS["BUS_NUM"][i] in 광역버스:
        BUS['광역버스'][i]=BUS['hap'][i]
    elif BUS["BUS_NUM"][i] in 지선버스:
        BUS['지선버스'][i]=BUS['hap'][i]
    elif BUS["BUS_NUM"][i] in 간선버스:
        BUS['간선버스'][i]=BUS['hap'][i]
    elif BUS["BUS_NUM"][i] in 마을버스:
        BUS['마을버스'][i]=BUS['hap'][i]
    else:
        print("잘못됨")

BUS=BUS[['NODENO','GPSLATI','GPSLONG','BRT','광역버스','지선버스','간선버스','마을버스']]

# gpslong 열에 ,가 한 데이터에 포함되어있어 삭제함.
BUS['GPSLONG']=BUS['GPSLONG'].astype(str)
BUS['GPSLONG'] = BUS['GPSLONG'].str.replace(',', '').astype('float64')

# groupby 함수 사용 위해 float으로 변환.
BUS['NODENO']=BUS['NODENO'].apply(pd.to_numeric)
BUS['GPSLATI']=BUS['GPSLATI'].apply(pd.to_numeric)
BUS['GPSLONG']=BUS['GPSLONG'].apply(pd.to_numeric)
BUS['BRT']=BUS['BRT'].apply(pd.to_numeric)
BUS['광역버스']=BUS['광역버스'].apply(pd.to_numeric)
BUS['지선버스']=BUS['지선버스'].apply(pd.to_numeric)
BUS['간선버스']=BUS['간선버스'].apply(pd.to_numeric)
BUS['마을버스']=BUS['마을버스'].apply(pd.to_numeric)

# 버스정류장번호(NODENO)별 
BUS1=BUS.groupby(["NODENO","GPSLATI","GPSLONG"])['BRT','광역버스','지선버스','간선버스','마을버스'].sum().reset_index()

# NODENO는 같지만, GPSLATI와 GPSLONG이 다른 데이터 존재.
BUS1=BUS1.drop_duplicates(['NODENO'], keep=False)

In [ ]:
bus = plus_grid(BUS1, geo, 'GPSLONG', 'GPSLATI')

### CCTV 데이터

In [ ]:
df = pd.read_csv('sejong_cctv.csv') # CCTV 데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

# CCTV 데이터에서 필요한 컬럼만 추출
df = df[['카메라대수','카메라화소수','위도','경도']]

cctv = plus_grid(df, geo, '경도', '위도')

### 가로등 데이터

In [ ]:
df = pd.read_csv('sejong_light.csv') # 가로등 데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

# 가로등 데이터에서 필요한 컬럼만 추출
df = df[['가로등위치명','소재지지번주소','위도','경도']]

street_lights = plus_grid(df, geo, '경도', '위도')

### 파출소 데이터

In [ ]:
df = pd.read_csv('sejong_policeoffice.csv') # 파출소 데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

# Hospital 데이터에서 필요한 컬럼만 추출
df = df[['name','입력주소','X','Y']]

police_office = plus_grid(df, geo, 'X', 'Y')

### 학교 데이터

In [ ]:
df = pd.read_csv('sejong_school.csv') # 학교 데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

# 학교 데이터에서 필요한 컬럼만 추출
df = df[['학교명','학교구분','주소','X','Y']]

school = plus_grid(df, geo, 'X', 'Y')

### 공원 데이터

In [ ]:
df = pd.read_csv('sejong_park.csv') #공원데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

# 공원 데이터에서 필요한 컬럼만 추출
df = df[['공원명','소재지지번주소','공원구분','위도','경도']]

parks = plus_grid(df, geo, '경도', '위도')

### 학원 데이터

In [ ]:
# 데이터를 불러옵니다
df = pd.read_csv('sejong_academy.csv') #학원데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

# 학원 데이터에서 필요한 컬럼만 추출
academy1 = df[['학원주소','X','Y']]

# 상권 데이터에서 학원 데이터를 추출합니다
academy2 = data14[data14['상권업종중분류명'].str.contains('학원')]
academy2 = academy2.rename({'lon':'X', 'lat':'Y', '도로명주소':'학원주소'}, axis='columns')
academy_cat = ['학원(종합)','학원-입시','학원-외국어/어학','학원-어린이영어','태권도장', '피아노/바이올린/기타','합기도장','서예/서화/미술']
academy2 = academy2[academy2.상권업종소분류명.isin(academy_cat)][['학원주소','X', 'Y']]

# 두 데이터를 합쳐줍니다
df = pd.merge(academy1, academy2, on=['X','Y','학원주소'], how='outer')

academy = plus_grid(df, geo, 'X', 'Y')

### 편의점 데이터

In [ ]:
# 데이터를 불러옵니다
df = data14

# 편의점 데이터만 추출합니다
df = df[df['상권업종소분류명'].str.contains('편의점')]
df = df[['행정동명','도로명주소','lon','lat']]

# 편의점 데이터를 격자 데이터에 부여
stores = plus_grid(df, geo, 'lon', 'lat')

### 은행 데이터

In [ ]:
df = pd.read_csv('sejong_bank.csv') #은행데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

df = df[['name', 'address', '경도','위도']]

# 은행 데이터를 격자 데이터에 부여
banks = plus_grid(df, geo, '경도', '위도')

### 약국 데이터

In [ ]:
df = pd.read_csv('sejong_pharmacy.csv') #약국데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

df = df[['_약국명칭', '약국소재지 도로명주소', 'X','Y']]

# 약국 데이터를 격자 데이터에 부여
pharmacy = plus_grid(df, geo, 'X', 'Y')

### 마트 데이터

In [ ]:
df = pd.read_csv('sejong_market.csv') #약국데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

df = df[['name', 'address', 'X','Y']]

# 약국 데이터를 격자 데이터에 부여
mart = plus_grid(df, geo, 'X', 'Y')

### 병원 데이터

In [ ]:
df = pd.read_csv('sejong_hospital.csv') # 병원 데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

# Hospital 데이터에서 필요한 컬럼만 추출
df = df[['name','입력주소','X','Y']]

hospital= plus_grid(df, geo, 'X', 'Y')

### 매매 데이터

In [ ]:
# 평당 가격 구하는 함수 정의
def price_per(df , col='거래금액(만원)'):
    df['평'] = df['전용면적(㎡)']/3.3
    df['거래금액'] = df[col].str.replace(',','').astype(int)
    df['평당가격'] = df['거래금액']/df['평']
    df = df.drop(['평',col], axis=1)
    return df

In [ ]:
# 매매, 전월세 데이터 geocoding 된 데이터
adr = pd.read_csv('geoaddress.csv')
adr = adr[['입력주소','X','Y']]

In [ ]:
df3 = price_per(data3)
df4 = price_per(data4)
df6 = price_per(data6)

In [ ]:
df3 = pd.merge(get_address(df3)[['계약년월','평당가격','입력주소','시군구']], adr, on='입력주소') #아파트 매매 데이터에 위경도 추가
apt_buy = plus_grid(df3, geo, 'X','Y', drop=False)

df4 = pd.merge(get_address(df4)[['계약년월','평당가격','입력주소','시군구']], adr, on='입력주소') #연립다세대 매매 데이터에 위경도 추가
vil_buy = plus_grid(df4, geo, 'X','Y', drop=False)

df6 = pd.merge(get_address(df6)[['계약년월','평당가격','입력주소','시군구']], adr, on='입력주소') #오피스텔 매매 데이터에 위경도 추가
ops_buy = plus_grid(df6, geo, 'X','Y', drop=False)

### 임대 데이터

In [ ]:
# 전월세 데이터를 전세와 월세로 구분합니다
# 1이 전세, 2가 월세

df71 = data7[data7['전월세구분'] == '전세']
df72 = data7[data7['전월세구분'] == '월세']
df81 = data8[data8['전월세구분'] == '전세']
df82 = data8[data8['전월세구분'] == '월세']
df101 = data10[data10['전월세구분'] == '전세']
df102 = data10[data10['전월세구분'] == '월세']

In [ ]:
df71 = price_per(df71, col='보증금(만원)')
df81 = price_per(df81, col='보증금(만원)')
df101 = price_per(df101, col='보증금(만원)')

In [ ]:
# 전세 데이터

df3 = pd.merge(get_address(df71)[['계약년월','평당가격','입력주소','시군구']], adr, on='입력주소') #아파트 매매 데이터에 위경도 추가
apt_full = plus_grid(df3, geo, 'X','Y', drop=False)

df4 = pd.merge(get_address(df81)[['계약년월','평당가격','입력주소','시군구']], adr, on='입력주소') #연립다세대 매매 데이터에 위경도 추가
vil_full = plus_grid(df4, geo, 'X','Y', drop=False)

df6 = pd.merge(get_address(df101)[['계약년월','평당가격','입력주소','시군구']], adr, on='입력주소') #오피스텔 매매 데이터에 위경도 추가
ops_full = plus_grid(df6, geo, 'X','Y', drop=False)

In [ ]:
# 임대(전세+월세) 데이터

df3 = pd.merge(get_address(data7)[['계약년월','입력주소','시군구']], adr, on='입력주소') #아파트 매매 데이터에 위경도 추가
apt_rent = plus_grid(df3, geo, 'X','Y', drop=False)

df4 = pd.merge(get_address(data8)[['계약년월','입력주소','시군구']], adr, on='입력주소') #연립다세대 매매 데이터에 위경도 추가
vil_rent = plus_grid(df4, geo, 'X','Y', drop=False)

df6 = pd.merge(get_address(data10)[['계약년월','입력주소','시군구']], adr, on='입력주소') #오피스텔 매매 데이터에 위경도 추가
ops_rent = plus_grid(df6, geo, 'X','Y', drop=False)

### 공시지가 데이터

In [ ]:
df = pd.read_csv('zin18.csv') #Geocoding 데이터
df = pd.merge(houseonly, df, left_on = '대지위치', right_on = '지번주소') # 공시지가 데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

df = df[['건물명', '지번주소','기준년도','공시지가', 'X','Y']]

# 공시지가 데이터를 격자 데이터에 부여
gongsi = plus_grid(df, geo, 'X', 'Y', drop=False)

### 표제부 주택 데이터

In [ ]:
# Geocoding된 주택 데이터
res = pd.concat([pd.read_csv('houseadd_file1.csv'), pd.read_csv('houseadd_file2.csv')])

# 주택 데이터를 격자 데이터에 부여
reals = plus_grid(res, geo, 'X', 'Y')

### 인구 데이터

In [ ]:
#거주인구정보 500미터 격자 파일과 합치기
grid_500 = geo[['id', 'geometry']]
merge19 = gpd.sjoin(grid_500, data19, op='intersects', how = 'left').reset_index()

# 중복 제거해주기
dropdu = merge19[['gid','201710_20대_거주인구수']].drop_duplicates().index
merge19 = merge19.loc[dropdu,:]
merge19_group = merge19.groupby('id').sum().reset_index()
population = merge19_group